## BiLSTM for MIT Movies

In [16]:
import os
import sys
import json
import numpy as np
from tqdm import tqdm
sys.path.append("..")

from torch import nn
from torch.optim import Adam
from src.movientities.models.lstm import BiLSTM
from src.movientities.trainers import TrainerBiLstm
from src.movientities.utils.processors import NerPreProcessor
from src.movientities.data.build_dataset import Corpus, BuildData

In [6]:
# For finding the absolute path dynamically for every other user for the sake of this notebooks paths
for path in globals()['_dh']:
    if "NamedEntityRecognizer" in path.split(os.sep):
        absolute_path = os.path.split(path)[0]
        break
print(absolute_path)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/home/karaz/anaconda3/envs/project/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-bab92a44302a>", line 1, in <module>
    absolute_path = [path for path in globals()['_dh'] if "NamedEntity" in path.split(os.sep)][0]
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/karaz/anaconda3/envs/project/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceb

In [13]:
# Create datasets with tab as splitter for corpus of torch text to handle - Uncomment if needed -
# Convert ->  O	good             -> to ->  good     O           
# Convert ->  B-GENRE	romantic -> to ->  romantic B-GENRE
# Convert ->  I-GENRE	comedies -> to ->  comedies I-GENRE
BuildData.create_finaldata(os.path.join(absolute_path, "data/raw/mitmovies/engtrain.bio"), os.path.join(absolute_path, "data/modified/mitmovies_tab_format/train.txt"), splits="\t")
BuildData.create_finaldata(os.path.join(absolute_path, "data/raw/mitmovies/engtest.bio"), os.path.join(absolute_path, "data/modified/mitmovies_tab_format/test.txt"), splits="\t")

In [14]:
# Dataset class
dataset = Corpus(
    input_folder=os.path.join(absolute_path, "data/modified/mitmovies_tab_format"),
    min_word_freq=3,  # any words occurring less than 3 times will be ignored from vocab
    batch_size=64)
print(f"Train set: {len(dataset.train_dataset)} sentences")
print(f"Test set: {len(dataset.test_dataset)} sentences")

Train set: 9775 sentences
Test set: 2443 sentences


In [10]:
model = BiLSTM(
    input_dim=len(dataset.word_field.vocab),
    embedding_dim=300,
    hidden_dim=64,
    output_dim=len(dataset.tag_field.vocab),
    lstm_layers=4,
    emb_dropout=0.25,
    lstm_dropout=0.01,
    fc_dropout=0.1,
    word_pad_idx=dataset.word_pad_idx,
)
# Initialize weights and embeddings
model.init_weights()
model.init_embeddings(word_pad_idx=dataset.word_pad_idx)
print(f"The model has {model.count_parameters():,} trainable parameters.")
print(model)

The model has 1,161,930 trainable parameters.
BiLSTM(
  (embedding): Embedding(2244, 300, padding_idx=1)
  (emb_dropout): Dropout(p=0.25, inplace=False)
  (lstm): LSTM(300, 64, num_layers=4, dropout=0.01, bidirectional=True)
  (fc_dropout): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=128, out_features=26, bias=True)
)


In [12]:
ner = TrainerBiLstm(
  model=model,
  data=dataset,
  optimizer_cls=Adam,
  loss_fn_cls=nn.CrossEntropyLoss,
  log_name="bilstm_vanilla2"
)
ner.train(20)

Epoch: 01 | Epoch Time: 0m 22s
	Trn Loss: 0.903 | Trn Acc: 75.58%
	Val Loss: 0.763 | Val Acc: 80.36% | Val Precision: 60.83% | Val Recall: 55.18% | Val F1 Macro: 53.84% | Val F1 Micro: 80.28%
Epoch: 02 | Epoch Time: 0m 22s
	Trn Loss: 0.448 | Trn Acc: 88.92%
	Val Loss: 0.520 | Val Acc: 87.07% | Val Precision: 74.27% | Val Recall: 72.57% | Val F1 Macro: 70.96% | Val F1 Micro: 86.40%
Epoch: 03 | Epoch Time: 0m 22s
	Trn Loss: 0.321 | Trn Acc: 92.06%
	Val Loss: 0.453 | Val Acc: 88.60% | Val Precision: 75.65% | Val Recall: 75.65% | Val F1 Macro: 73.28% | Val F1 Micro: 87.79%
Epoch: 04 | Epoch Time: 0m 21s
	Trn Loss: 0.272 | Trn Acc: 93.18%
	Val Loss: 0.432 | Val Acc: 88.86% | Val Precision: 77.34% | Val Recall: 73.99% | Val F1 Macro: 73.20% | Val F1 Micro: 87.94%
Epoch: 05 | Epoch Time: 0m 22s
	Trn Loss: 0.238 | Trn Acc: 94.04%
	Val Loss: 0.415 | Val Acc: 89.34% | Val Precision: 74.38% | Val Recall: 74.84% | Val F1 Macro: 72.41% | Val F1 Micro: 88.23%
Epoch: 06 | Epoch Time: 0m 21s
	Trn Loss